In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model,Sequential,load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import csv
import os
from IPython.display import clear_output
import re
import unicodedata

2024-08-31 07:17:50.897445: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-31 07:17:50.897518: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-31 07:17:50.899228: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
tf.__version__

'2.15.0'

In [ ]:
!nvidia-smi

In [ ]:
#Si se va a usar tpu

tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

In [3]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿0-9]+", " ", w)

    w = w.strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    return w

In [9]:
file = open("/kaggle/input/bbc-news-articles/bbc_news_20220307_20240703.csv","r")
reader = csv.reader(file)
entrada_texto = []
salida_texto = []
ist_first = True
for row in reader:
    if ist_first:
        ist_first = False
        continue
    entrada_texto.append(row[0])
    salida_texto.append(row[4])
    

size = None
begin_token = "<sos>"
end_token = "<eos>"


entrada_texto_p = [
    "{} {} {}".format(
        begin_token,
        preprocess_sentence(text.lower()),
        end_token) for text in entrada_texto[:size]]
salida_texto_p = [
    "{} {} {}".format(
        begin_token,
        preprocess_sentence(text.lower()),
        end_token) for text in salida_texto[:size]]


In [10]:
print(entrada_texto_p[0])
print("-"*20)
print(salida_texto_p[0])

<sos> ukraine angry zelensky vows to punish russian atrocities <eos>
--------------------
<sos> the ukrainian president says the country will not forgive or forget those who murder its civilians . <eos>


In [11]:
## Parámetros para el tokenizador
max_len = 100
filters = '!"#$%&()*+-/:?@[\\]^_`{|}~\t¡’’'

tokenizer = Tokenizer(num_words=None, oov_token='<OOV>',filters=filters)
tokenizer.fit_on_texts(entrada_texto_p,)
tokenizer.fit_on_texts(salida_texto_p)


In [ ]:
#Este codigo es si vas a cargar checkpoints
import json
from tensorflow.keras.preprocessing.text import tokenizer_from_json
f1 = open("tokenizer.json")
max_len = 600
filters = '!"#$%&()*+-/:?@[\\]^_`{|}~\t¡’’'
json1 = json.load(f1)
tokenizer = tokenizer_from_json(json1)

In [12]:
vocab_size=len(tokenizer.word_index)
print(vocab_size)

34019


In [13]:
sequences_input = tokenizer.texts_to_sequences(entrada_texto_p)

In [14]:
padded_input = pad_sequences(sequences_input)
sequences_output = tokenizer.texts_to_sequences(salida_texto_p)
padded_output = pad_sequences(sequences_output,maxlen=max_len+1)


In [15]:
print(len(padded_input))

35860


In [16]:
batch_size=16
# Crear un objeto Dataset a partir de los datos tokenizados
dataset = tf.data.Dataset.from_tensor_slices((padded_input, padded_output))
dataset = dataset.batch(batch_size).shuffle(1024)

In [17]:
for x,y in dataset.take(1):
    print(x.shape,y.shape)
    x0,y0 = np.array([x[0]]).tolist(),np.array([y[0]]).tolist()
    print(x0,y0)
    print(tokenizer.sequences_to_texts(x0),tokenizer.sequences_to_texts(y0))
    
print(len(tokenizer.word_index))

(16, 28) (16, 101)
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 864, 89, 87, 168, 400, 174, 343, 6, 343, 7, 251, 606, 3]] [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1408, 22, 331, 210, 11, 147, 1108, 643, 26, 606, 406, 14, 4, 42, 14, 563, 5, 3]]
['<OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <sos> ni election 2022 party leaders go head to head in tv debate <eos>'] ['<OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV

In [23]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)

In [24]:
def scaled_dot_product_attention(q, k, v, mask):
  
    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

In [25]:
@tf.keras.utils.register_keras_serializable()
class SelfMultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, num_heads , d_model, dropout_rate):
        super(SelfMultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // self.num_heads

        self.wq = Dense(d_model)
        self.wk = Dense(d_model)
        self.wv = Dense(d_model)

        self.dense = Dense(d_model)
        self.drop = Dropout(dropout_rate)
        self.drop1 = Dropout(dropout_rate)
        self.drop2 = Dropout(dropout_rate)
        self.drop3 = Dropout(dropout_rate)
        
    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, attention_mask):
        batch_size = tf.shape(q)[0]

        q = self.drop1(self.wq(q))  # (batch_size, seq_len, d_model)
        k = self.drop2(self.wk(k))  # (batch_size, seq_len, d_model)
        v = self.drop3(self.wv(v))  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, attention_mask)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)
        
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        output = self.drop(output)
        return output

In [26]:
@tf.keras.utils.register_keras_serializable()
class FeedForward(Layer):
    def __init__(self, d_model, dff, dropout_rate=0.1):
        super().__init__()
        self.seq = tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model),
        tf.keras.layers.Dropout(dropout_rate)
        ])
        self.add = tf.keras.layers.Add()
        self.layer_norm = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    def call(self, x):
        x = self.add([x, self.seq(x)])
        x = self.layer_norm(x)
        return x


In [27]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

In [28]:
def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

In [29]:
@tf.keras.utils.register_keras_serializable()
class EncoderLayer(tf.keras.Layer):
    def __init__(self,dim,n_heads,dff,dropout_rate=0.1):
        super(EncoderLayer,self).__init__()
        self.mha = SelfMultiHeadAttention(n_heads,dim,dropout_rate=dropout_rate)
        self.ffn = FeedForward(dim,dff,dropout_rate)
        self.norm1 = LayerNormalization(epsilon=1e-6)
        self.norm2 = LayerNormalization(epsilon=1e-6)
    def build(self, input_shape):
        super(EncoderLayer, self).build(input_shape)

    def call(self,x,training=True,src_padding_mask=None):
        out1 = self.mha(x,x,x,attention_mask=src_padding_mask,training=training)
        out1 = self.norm1(out1+x)
        out2 = self.ffn(out1)
        out2 = self.norm2(out1+out2)
        return out2

In [30]:
@tf.keras.utils.register_keras_serializable()
class DecoderLayer(tf.keras.Layer):
    def __init__(self,dim,n_heads,dff,dropout_rate=0.1):
        super(DecoderLayer,self).__init__()
        self.mha1 = SelfMultiHeadAttention(n_heads,dim,dropout_rate=dropout_rate)
        self.mha2 = SelfMultiHeadAttention(n_heads,dim,dropout_rate=dropout_rate)
        self.norm1 = LayerNormalization(epsilon=1e-6)
        self.norm2 = LayerNormalization(epsilon=1e-6)
        self.norm3 = LayerNormalization(epsilon=1e-6)
        self.ffn = FeedForward(dim,dff,dropout_rate)
    def build(self, input_shape):
        super(DecoderLayer, self).build(input_shape)

    def call(self,x,enc_output,look_ahead_mask=None,padding_mask=None):
        out1 = self.mha1(x,x,x,attention_mask=look_ahead_mask)
        out1 = self.norm1(x+out1)
        out2 = self.mha2(enc_output,enc_output,out1,attention_mask=padding_mask)
        out2 = self.norm2(out1+out2)
        out3 = self.ffn(out2)
        out3 = self.norm3(out3+out2)
        return out3

In [31]:
@tf.keras.utils.register_keras_serializable()
class Encoder(tf.keras.Layer):
    def __init__(self,dim,n_layers,n_heads,dff,vocab_size,dropout_rate=0.1,length=2048):
        super(Encoder,self).__init__()
        self.n_layers = n_layers
        self.dim = dim
        self.embedding = Embedding(vocab_size,dim)
        self.enc_layers = [
            EncoderLayer(dim,n_heads,dff,dropout_rate) for _ in range(n_layers)
        ]
        self.drop=Dropout(dropout_rate)
        self.pos_encoding = positional_encoding(length,
            self.dim)
        self.dropout=Dropout(dropout_rate)
    def build(self, input_shape):
        super(Encoder, self).build(input_shape)

    def call(self,x,src_padding_mask=None,training=None):
        seq_len = tf.shape(x)[1]

        # adding embedding and position encoding.
        x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.dim, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.n_layers):
            x = self.enc_layers[i](x,src_padding_mask=src_padding_mask)
        x = self.drop(x)
        return x

In [32]:
@tf.keras.utils.register_keras_serializable()
class Decoder(tf.keras.Layer):
    def __init__(self,dim,n_layers,n_heads,dff,vocab_size,dropout_rate=0.1,length=2048):
        super(Decoder,self).__init__()
        self.n_layers = n_layers
        self.dim = dim
        self.embedding = tf.keras.layers.Embedding(vocab_size, dim)
        self.pos_encoding = positional_encoding(length, dim)
        self.dec_layers = [
            DecoderLayer(dim,n_heads,dff,dropout_rate) for _ in range(n_layers)
        ]
        self.dropout = Dropout(dropout_rate)
    def build(self, input_shape):
        super(Decoder, self).build(input_shape)

    def call(self,x,enc_out,look_ahead_mask=None,tgt_padding_mask=None,training=None):
        seq_len = tf.shape(x)[1]
        x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.dim, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x,training=training)
        for i in range(self.n_layers):
            x = self.dec_layers[i](
                x,
                enc_out,
                look_ahead_mask=look_ahead_mask,
                padding_mask=tgt_padding_mask)
        return x

In [33]:
@tf.keras.utils.register_keras_serializable()
class Transformer(tf.keras.Model):
    def __init__(self,
                 dim, enc_layers,
                 dec_layers, heads,
                 dff,
                 vocab_size,
                 dropout_rate=0.1,
                 ilen=2048, tlen=2048,**kwargs):
        super(Transformer, self).__init__()
        self.name = ""
        self.dim = dim
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
        self.dff = dff
        self.heads = heads
        self.vocab_size = vocab_size
        self.dropout_rate = dropout_rate
        self.ilen = ilen
        self.tlen = tlen
        self.encoder = Encoder(
            self.dim, self.enc_layers, self.heads,
            self.dff, self.vocab_size, dropout_rate=self.dropout_rate, length=self.ilen)
        self.decoder = Decoder(self.dim, self.dec_layers, self.heads, self.dff,self.vocab_size,
                               dropout_rate=self.dropout_rate, length=self.tlen)
        self.linear = Dense(self.vocab_size)
    
    def get_config(self):
        config = super(Transformer, self).get_config()
        config.update({
            'dim': self.encoder.dim,   # Asumiendo que estos son atributos de Encoder y Decoder
            'enc_layers': self.enc_layers,
            'dec_layers': self.dec_layers,
            'heads': self.heads,
            'dff': self.dff,
            'vocab_size': self.vocab_size,
            'dropout_rate': self.dropout_rate,
            'ilen': self.ilen,  # Assuming `length` is the length of the encoder
            'tlen': self.tlen,  # Assuming `length` is the length of the decoder
        })
        return config
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

    def build(self,input_shape):
        super(Transformer,self).build(input_shape)
    
    def call(self, inputs, src_padding_mask=None, look_ahead_mask=None, tgt_padding_mask=None, training=None):
        context, x = inputs
        enc_out = self.encoder(context, src_padding_mask, training=training)
        dec_out = self.decoder(x, enc_out,
                               look_ahead_mask=look_ahead_mask,
                               tgt_padding_mask=tgt_padding_mask,
                               training=training)
        logits = self.linear(dec_out)

        try:
            del logits.keras_mask
        except AttributeError:
            pass
        return logits


In [34]:
using_tpu=False

In [35]:
dim=512
enc_layers=4
dec_layers=4
heads=8
dff=2048
dropout_rate=0.1
ilen=500
tlen=5000
if using_tpu:
    with tpu_strategy.scope():
        transformer = Transformer(dim,
                                  enc_layers,
                                  dec_layers,
                                  heads,
                                  dff,
                                  vocab_size,
                                  dropout_rate=dropout_rate,
                                  ilen=ilen,
                                  tlen=tlen)
else:
    transformer = Transformer(dim,
                                  enc_layers,
                                  dec_layers,
                                  heads,
                                  dff,
                                  vocab_size,
                                  dropout_rate=dropout_rate,
                                  ilen=ilen,
                                  tlen=tlen)
    input_shape = (1, ilen)
    transformer.build(input_shape)

In [46]:
transformer.summary()

Model: ""

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder (Encoder)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Decoder)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [47]:
input_shape = (1, ilen)
transformer.build(input_shape)

In [37]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [38]:
def create_masks(inp, tar):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(inp)

    dec_padding_mask = create_padding_mask(inp)
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask

In [ ]:

example_input = (tf.zeros((1, ilen)),
                 tf.zeros((1, tlen)))
a,b,c = create_masks(example_input[0],example_input[1])
transformer(example_input,a,b,c)
print()

In [ ]:
transformer.load_weights("wr.weights.h5")

In [48]:
@tf.keras.utils.register_keras_serializable()
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


In [49]:
if using_tpu:
    with tpu_strategy.scope():
        learning_rate = CustomSchedule(dim,500)
        optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                             epsilon=1e-9)
else:
    learning_rate = CustomSchedule(dim,500)
    optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                             epsilon=1e-9)

In [50]:
ckpt_path = "./"
ckpt_prefix = os.path.join(ckpt_path,"ckpt")
ckpt = tf.train.Checkpoint(
    optimizer=optimizer)

In [ ]:
optimizer.build(transformer.trainable_variables)

In [51]:
transformer.trainable_variables

[]

In [ ]:
ckpt_manager = tf.train.CheckpointManager(ckpt, ckpt_path, max_to_keep=5)

In [ ]:
ckpt.save(ckpt_prefix)

In [ ]:
ckpt_manager.restore_or_initialize()

In [ ]:
ckpt.restore("ckpt-1")

In [53]:
if using_tpu:
    with tpu_strategy.scope():
        loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
else:
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [52]:
def masked_loss(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)
    #return tf.reduce_mean(loss_)

def masked_accuracy(label, pred):
    pred = tf.argmax(pred, axis=2)
    label = tf.cast(label, pred.dtype)
    match = label == pred

    mask = label != 0

    match = match & mask

    match = tf.cast(match, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(match)/tf.reduce_sum(mask)


In [54]:
@tf.function()
def train_step(inp,tg,clip_value=1.0):
    tg_m = tg[:,:-1]
    tg_f = tg[:,1:]
    s = tf.shape(tg_m)
    
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp,tg_m)
    with tf.GradientTape() as tape:
        pred = transformer(
        inputs=(inp, tg_m),
        training=True,
        src_padding_mask=enc_padding_mask,
        look_ahead_mask=combined_mask,
        tgt_padding_mask=dec_padding_mask
        )

        loss = masked_loss(tg_f,pred)
        grads = tape.gradient(loss,transformer.trainable_variables)

    #grads = [tf.clip_by_value(grad, -clip_value, clip_value) for grad in grads]
    optimizer.apply_gradients(zip(grads,transformer.trainable_variables))
    batch_acc = masked_accuracy(tg_f,pred)
    return loss,batch_acc

In [55]:
epochs = 10
for epoch in range(epochs):
    print("Epoch {}".format(epoch+1))
    n = 0
    l=0
    ac=0
    for inp,tg in dataset:
        if using_tpu:
            la,acc = tpu_strategy.gather(tpu_strategy.run(train_step,args=(inp,tg)),axis=0)
        else:
            la,acc = train_step(inp,tg)
        l += la
        ac+=acc
        n +=1
        
        print(".",end="")
    print()
    print("Loss {} \nAccuracy {}".format(l/n,ac/n))

Epoch 1
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

KeyboardInterrupt: 

In [ ]:
for inp,tg in dataset.take(1):
    inp_m = inp
    tg_m = tg[:,:-1]
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp_m,tg_m)
    #tpreds = tpu_strategy.run(tpu_transformer_pred,
    #                          args=((inp_m,tg_m),
    #                                enc_padding_mask,
    #                                combined_mask, 
    #                                dec_padding_mask))
    #tpreds = np.array(tpu_strategy.gather(tpreds, axis=0))
    pred = transformer((inp_m,tg_m),
                       src_padding_mask=enc_padding_mask,
                       look_ahead_mask=combined_mask,
                       tgt_padding_mask=dec_padding_mask)
    preds = [int(tf.argmax(p)) for p in pred[0]]
    print(tokenizer.sequences_to_texts([np.array(inp[0])]))
    #print(preds)
    #print(tg[0])
    print(tokenizer.sequences_to_texts([preds]))
    print(tokenizer.sequences_to_texts([np.array(tg[0])]))

In [ ]:
def predict_with_transformer(model,text, max_len=20):
    text_with_start = f"{begin_token} {text.lower()} {end_token}"
    
    sequence_input = tokenizer.texts_to_sequences([text_with_start])[0]
    print(sequence_input)
    encoder_input = tf.expand_dims(sequence_input,0)
    
    decoder_input = [tokenizer.word_index.get(begin_token, 0)]
    output = tf.expand_dims(decoder_input,0)
    for step in range(max_len):
        # Preparar los inputs para el modelo
        # Crear las máscaras necesarias
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

        # Obtener logits del modelo
        logits = model((encoder_input,output), training=False,
                             src_padding_mask=enc_padding_mask,
                             look_ahead_mask=combined_mask,
                             tgt_padding_mask=dec_padding_mask)

        
        logits_for_last_token = logits[:, -1, :]  # (batch_size, vocab_size)
        #print("Dimensiones de logits_for_last_token:", logits_for_last_token.shape)

        predicted_id = tf.cast([tf.argmax(logits_for_last_token,axis=-1)],tf.int32)
        #print(predicted_id)
        #print(output)
        if predicted_id == tokenizer.word_index.get(end_token, 0):
            return tf.squeeze(output, axis=0)

        output = tf.concat([output, predicted_id], axis=-1)

        print(tokenizer.sequences_to_texts([[tf.squeeze(predicted_id).numpy()]])[0] +" ",end="")
        
    # Convertir los tokens a texto utilizando el tokenizer de salida
    predicted_sequence = tokenizer.sequences_to_texts([tf.squeeze(output).numpy()])
        
    # Mostrar la secuencia de tokens generados para depuración
    #print(f"Tokens generados: {decoder_input}")
    
    return predicted_sequence

In [ ]:
predict_with_transformer(transformer," chat gpt is smarter than humans ",50)

**Guardamos el tokenizador y el modelo**

In [ ]:
!rm tokenizer.json

In [ ]:
import io
import json
#Guardamos tokenizador
inp_t_json = tokenizer.to_json()
with io.open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(inp_t_json, ensure_ascii=False))
    

In [ ]:
print(tf.config.list_physical_devices())

In [ ]:
with tf.device("CPU"):
    transformer2 = Transformer(dim,
                                  enc_layers,
                                  dec_layers,
                                  heads,
                                  dff,
                                  vocab_size,
                                  dropout_rate=dropout_rate,
                                  ilen=ilen,
                                  tlen=tlen)
    
    
    example_input = (tf.zeros((1, ilen)),
                     tf.zeros((1, tlen)))
    a,b,c = create_masks(example_input[0],example_input[1])
    transformer2(example_input,a,b,c)
    
    transformer2.set_weights(transformer.get_weights())

In [ ]:
#tf.keras.saving.save_model(transformer,"transformer.h5")
transformer2.save_weights("wr.weights.h5")

In [ ]:
transformer2.summary()

In [ ]:
transformer2.load_weights("w.weights.h5")

In [ ]:
predict_with_transformer(transformer2," good. what are you doing ",100)

In [ ]:
ckpt.save(ckpt_prefix)

In [ ]:
from IPython.display import FileLink
import os

In [ ]:
names = os.listdir(ckpt_path)
names

In [ ]:
FileLink(names[-3])

In [ ]:
FileLink("checkpoints/"+names[1])

In [ ]:
FileLink("checkpoints/"+names[2])

In [ ]:
loaded_model = tf.keras.models.load_model("transformer.keras")